In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')
from src.encode_data import *
from src.midi_data import *
from src.data_sources import process_all, arr2csv
from src.midi_transform import *
from src.fastai_data import *

In [3]:
import traceback
import time

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v16'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
# sf_path = f'sf{SAMPLE_FREQ}'
sf_path = f'sf{12}'
# dir_name = 'piano_duet'
# dir_name = 'midi_encode'
dir_name = 's2s_encode'
out_dir = Path(f'{sf_path}/{dir_name}')

In [7]:
out_csv = version_path/out_dir/f'{dir_name}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
out_csv

PosixPath('data/midi/v16/sf12/s2s_encode/s2s_encode.csv')

### Encoding midi to numpy

In [8]:
df = pd.read_csv(out_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,4,5,9,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,artist,genres,mxl,ht_bpm,ht_key,parts,md5,numpy,midi,section,midi_title,ht_time_signature,ht_mode,ht_offset,source,title,song_url
0,wayne-sharpe,NaN,NaN,128.0,C,"intro,chorus",bf1f29e5ff84e3e93e37fb873bfb590e,s2s_encode/hooktheory/pianoroll/w/wayne-sharpe...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,chorus,yu-gi-oh3,4.0,1.0,0.0,hooktheory,yu-gi-oh-theme-song,https://www.hooktheory.com/theorytab/view/wayn...
1,wayne-sharpe,NaN,NaN,85.0,C,"intro,chorus",055f80ad67f64edb14a85ca8fbfe8c29,NaN,midi_sources/hooktheory/pianoroll/w/wayne-shar...,intro,yu-gi-oh,3.0,1.0,0.0,hooktheory,yu-gi-oh-theme-song,https://www.hooktheory.com/theorytab/view/wayn...
2,what-a-day,Jazz,NaN,96.0,D,chorus,197f96f5d181f6ce1e2c5ab04ac1ff87,s2s_encode/hooktheory/pianoroll/w/what-a-day/k...,midi_sources/hooktheory/pianoroll/w/what-a-day...,chorus,kiefer,4.0,6.0,-5.0,hooktheory,kiefer,https://www.hooktheory.com/theorytab/view/what...
3,whiteflame,"J-Pop,Pop",NaN,152.0,D,"verse,pre-chorus,chorus",9e7ce13a35f1314423a9a6d5a5287a4a,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...,midi_sources/hooktheory/pianoroll/w/whiteflame...,pre-chorus,senbonzakura - pre-Pre-Chorus,4.0,6.0,-5.0,hooktheory,senbonzakura,https://www.hooktheory.com/theorytab/view/whit...
4,whiteflame,"J-Pop,Pop",NaN,152.0,D,"verse,pre-chorus,chorus",d5aaf79d0989222f1362f9f46c540a27,s2s_encode/hooktheory/pianoroll/w/whiteflame/s...,midi_sources/hooktheory/pianoroll/w/whiteflame...,verse,Senbonzakura,4.0,6.0,-5.0,hooktheory,senbonzakura,https://www.hooktheory.com/theorytab/view/whit...


In [9]:
for i, row in df.iterrows():
    val = row.numpy
    if isinstance(val, str) and val.startswith(f'{dir_name}/'):
        new_val = f'{sf_path}/'+val
    df.at[i,'numpy'] = new_val

In [10]:
df

,artist,genres,mxl,ht_bpm,ht_key,parts,md5,numpy,midi,section,midi_title,ht_time_signature,ht_mode,ht_offset,source,title,song_url
0,wayne-sharpe,NaN,NaN,128.0,C,"intro,chorus",bf1f29e5ff84e3e93e37fb873bfb590e,sf12/s2s_encode/hooktheory/pianoroll/w/wayne-s...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,chorus,yu-gi-oh3,4.0,1.0,0.0,hooktheory,yu-gi-oh-theme-song,https://www.hooktheory.com/theorytab/view/wayn...
1,wayne-sharpe,NaN,NaN,85.0,C,"intro,chorus",055f80ad67f64edb14a85ca8fbfe8c29,sf12/s2s_encode/hooktheory/pianoroll/w/wayne-s...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,intro,yu-gi-oh,3.0,1.0,0.0,hooktheory,yu-gi-oh-theme-song,https://www.hooktheory.com/theorytab/view/wayn...
2,what-a-day,Jazz,NaN,96.0,D,chorus,197f96f5d181f6ce1e2c5ab04ac1ff87,sf12/s2s_encode/hooktheory/pianoroll/w/what-a-...,midi_sources/hooktheory/pianoroll/w/what-a-day...,chorus,kiefer,4.0,6.0,-5.0,hooktheory,kiefer,https://www.hooktheory.com/theorytab/view/what...
3,whiteflame,"J-Pop,Pop",NaN,152.0,D,"verse,pre-chorus,chorus",9e7ce13a35f1314423a9a6d5a5287a4a,sf12/s2s_encode/hooktheory/pianoroll/w/whitefl...,midi_sources/hooktheory/pianoroll/w/whiteflame...,pre-chorus,senbonzakura - pre-Pre-Chorus,4.0,6.0,-5.0,hooktheory,senbonzakura,https://www.hooktheory.com/theorytab/view/whit...
4,whiteflame,"J-Pop,Pop",NaN,152.0,D,"verse,pre-chorus,chorus",d5aaf79d0989222f1362f9f46c540a27,sf12/s2s_encode/hooktheory/pianoroll/w/whitefl...,midi_sources/hooktheory/pianoroll/w/whiteflame...,verse,Senbonzakura,4.0,6.0,-5.0,hooktheory,senbonzakura,https://www.hooktheory.com/theorytab/view/whit...
5,whiteflame,"J-Pop,Pop",NaN,152.0,D,"verse,pre-chorus,chorus",e0c189ee753b30c4758d85211f13c189,sf12/s2s_encode/hooktheory/pianoroll/w/whitefl...,midi_sources/hooktheory/pianoroll/w/whiteflame...,chorus,Senbonzakura,4.0,6.0,-5.0,hooktheory,senbonzakura,https://www.hooktheory.com/theorytab/view/whit...
6,wham,Holiday,NaN,108.0,Db,"intro,verse,chorus",38e38402443506e326b76536e8e327a0,sf12/s2s_encode/hooktheory/pianoroll/w/wham/la...,midi_sources/hooktheory/pianoroll/w/wham/last-...,verse,Last Christmas Verse,4.0,1.0,-1.0,hooktheory,last-christmas,https://www.hooktheory.com/theorytab/view/wham...
7,wham,Holiday,NaN,108.0,Db,"intro,verse,chorus",75d0251177c8c1fa9a02821299fa5ba8,sf12/s2s_encode/hooktheory/pianoroll/w/wham/la...,midi_sources/hooktheory/pianoroll/w/wham/last-...,chorus,Last Christmas Chorus,4.0,1.0,-1.0,hooktheory,last-christmas,https://www.hooktheory.com/theorytab/view/wham...
8,wham,Holiday,NaN,108.0,Db,"intro,verse,chorus",83d2a800f40aeca07e30e4718cda8fe5,sf12/s2s_encode/hooktheory/pianoroll/w/wham/la...,midi_sources/hooktheory/pianoroll/w/wham/last-...,intro,Last Christmas Intro,4.0,1.0,-1.0,hooktheory,last-christmas,https://www.hooktheory.com/theorytab/view/wham...
9,wham,NaN,NaN,128.0,C,chorus,60fa29cfec107df27b053cf9708823d5,sf12/s2s_encode/hooktheory/pianoroll/w/wham/fr...,midi_sources/hooktheory/pianoroll/w/wham/freed...,chorus,Freedom Chorus,4.0,1.0,0.0,hooktheory,freedom,https://www.hooktheory.com/theorytab/view/wham...


In [11]:
df.to_csv(out_csv, index=False)